<a href="https://colab.research.google.com/github/alirezasdg1/Amazon-recommender-system/blob/master/notebooks/NN_TextReview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df_all_15 = pd.read_csv("/content/drive/My Drive/Datasets/df_all_15_text.csv")

In [4]:
df_all_15

,Unnamed: 0,fit,title,also_buy,image_x,price,asin,overall,reviewerID,reviewText,item_text,title_nlp
0,0,NaN,Black Mountain Products Resistance Band Set wi...,"['1612431712', 'B01AVDVHTI', 'B002YQUP7Q', 'B0...",['https://images-na.ssl-images-amazon.com/imag...,$17.32,7245456313,5.0,A1J5VYG8O0YEMB,work advertis work advertis,black mountain,black mountain product resist band set door an...
1,1,NaN,Black Mountain Products Resistance Band Set wi...,"['1612431712', 'B01AVDVHTI', 'B002YQUP7Q', 'B0...",['https://images-na.ssl-images-amazon.com/imag...,$17.32,7245456313,4.0,AYMDRRCOPKTCF,four star product arriv prompt describ,black mountain,black mountain product resist band set door an...
2,2,NaN,Black Mountain Products Resistance Band Set wi...,"['1612431712', 'B01AVDVHTI', 'B002YQUP7Q', 'B0...",['https://images-na.ssl-images-amazon.com/imag...,$17.32,7245456313,4.0,A3AX86YSUHLELL,great resist band top qualiti okay resist band...,black mountain,black mountain product resist band set door an...
3,3,NaN,Black Mountain Products Resistance Band Set wi...,"['1612431712', 'B01AVDVHTI', 'B002YQUP7Q', 'B0...",['https://images-na.ssl-images-amazon.com/imag...,$17.32,7245456313,5.0,A12MJJT1RL2S7K,perfect transport resist band perfect cross fi...,black mountain,black mountain product resist band set door an...
4,4,NaN,Black Mountain Products Resistance Band Set wi...,"['1612431712', 'B01AVDVHTI', 'B002YQUP7Q', 'B0...",['https://images-na.ssl-images-amazon.com/imag...,$17.32,7245456313,5.0,A33BKACRGR3H38,awesom great band crosstrain dont like lift gy...,black mountain,black mountain product resist band set door an...
...,...,...,...,...,...,...,...,...,...,...,...,...
227504,228653,NaN,Gas ONE GS-3000 Portable Gas Stove with Carryi...,"['B000JORB2M', 'B001D7FYCI', 'B00JT8ZYCS', 'B0...",['https://images-na.ssl-images-amazon.com/imag...,$5.12,B01HHB2HK0,5.0,A1RL3SGTCTKOVA,great stove got mine price time figur could go...,gason,ga one portabl ga stove carri case btu csa app...
227505,228654,NaN,Gas ONE GS-3000 Portable Gas Stove with Carryi...,"['B000JORB2M', 'B001D7FYCI', 'B00JT8ZYCS', 'B0...",['https://images-na.ssl-images-amazon.com/imag...,$5.12,B01HHB2HK0,5.0,A1AZ1NXGGP6LDX,great emerg sturdi use extra burner parti powe...,gason,ga one portabl ga stove carri case btu csa app...
227506,228655,NaN,Gas ONE GS-3000 Portable Gas Stove with Carryi...,"['B000JORB2M', 'B001D7FYCI', 'B00JT8ZYCS', 'B0...",['https://images-na.ssl-images-amazon.com/imag...,$5.12,B01HHB2HK0,5.0,A1S08PNPO3HXA5,good solid heat cheap fuel good price stove ac...,gason,ga one portabl ga stove carri case btu csa app...
227507,228656,NaN,Gas ONE GS-3000 Portable Gas Stove with Carryi...,"['B000JORB2M', 'B001D7FYCI', 'B00JT8ZYCS', 'B0...",['https://images-na.ssl-images-amazon.com/imag...,$5.12,B01HHB2HK0,5.0,A2WVF9ZQ068DN0,outstand conveni portabl cookstov own decad ab...,gason,ga one portabl ga stove carri case btu csa app...


In [5]:
df_items_text = df_all_15.groupby(['asin', 'title','title_nlp','image_x'])['item_text'].apply(lambda x: ' '.join(x)).reset_index()

In [6]:
df_review_text = df_all_15.groupby(['asin', 'title','title_nlp','image_x'])['reviewText'].apply(lambda x: ' '.join(x)).reset_index()

In [7]:
merged_df = df_items_text.merge(df_review_text, how='inner', on=['asin', 'title','title_nlp','image_x'])

In [10]:
merged_df.head()

,asin,title,title_nlp,image_x,item_text,reviewText
0,7245456275,Black Mountain Products Single Resistance Band...,black mountain product singl resist band door ...,['https://images-na.ssl-images-amazon.com/imag...,black mountain color black black mountain colo...,four star great band price door attach work wo...
1,7245456313,Black Mountain Products Resistance Band Set wi...,black mountain product resist band set door an...,['https://images-na.ssl-images-amazon.com/imag...,black mountain black mountain black mountain b...,work advertis work advertis four star product ...
2,B00003G4JR,Carrom Game Board Large,carrom game board larg,['https://images-na.ssl-images-amazon.com/imag...,carrom carrom carrom carrom carrom carrom,well made classic game assort excel build qual...
3,B00004NKIQ,Franklin Sports Adjustable Soccer Rebounder,franklin sport adjust soccer rebound,['https://images-na.ssl-images-amazon.com/imag...,franklin sport style name x black franklin spo...,perfect grandchildren perfect grandchildren bo...
4,B00004SYN6,Park &amp; Sun Sports Permanent Outdoor Tether...,park amp sun sport perman outdoor tetherbal se...,['https://images-na.ssl-images-amazon.com/imag...,park sun sport style name pole park sun sport ...,much better glu tablet grandson love much bett...


In [11]:
merged_df["reviewText"].iloc[:4]

0    four star great band price door attach work wo...
1    work advertis work advertis four star product ...
2    well made classic game assort excel build qual...
3    perfect grandchildren perfect grandchildren bo...
Name: reviewText, dtype: object

In [14]:
review_embeddings = hub.text_embedding_column("product_reviews",module_spec="https://tfhub.dev/google/universal-sentence-encoder/2")

In [15]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/Wiki-words-250-with-normalization/2")
embeddings = embed(merged_df["reviewText"])


In [26]:
embeddings.numpy()

(7119, 250)

In [17]:
hub_layer = hub.KerasLayer("https://tfhub.dev/google/Wiki-words-250-with-normalization/2",
                           input_shape=[], dtype=tf.string)

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 250)               252343750 
_________________________________________________________________
dense (Dense)                (None, 16)                4016      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 252,347,783
Trainable params: 4,033
Non-trainable params: 252,343,750
_________________________________________________________________


In [28]:
df_embeddings = pd.DataFrame(embeddings.numpy())

In [31]:
df_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249
0,-1.004372,-0.252154,0.099490,-0.777266,-0.047025,0.713028,-0.158967,-0.549558,-0.060963,0.095891,-0.409758,1.803299,-0.212956,-0.455865,0.404306,0.274597,0.234470,-0.065960,-0.206191,0.036072,0.536143,0.899337,0.186523,-0.226335,1.078249,-0.510379,0.110155,0.910803,0.712520,0.834203,1.710406,-0.438271,0.457348,-0.351890,0.148772,0.901526,-0.710523,0.062128,0.758749,0.004921,...,-0.501089,0.843945,1.708416,-1.160044,1.844806,0.253354,0.100809,0.389314,0.167228,-0.086408,-1.097299,0.498713,0.532329,0.451947,-0.478278,-0.160028,-0.344723,-0.644835,-0.586698,-0.901778,0.465793,-0.698533,-0.584918,0.783365,0.069082,0.255049,-0.343278,-0.129325,-0.329788,-0.503793,-0.050138,1.353157,0.108873,0.492900,0.370604,-0.401721,-0.126525,-0.156951,0.288336,0.168903
1,-2.322126,-0.576584,0.552689,-1.649610,-0.236413,1.775368,-0.479612,-1.274916,-0.355722,0.353430,-0.880688,4.580889,-0.298576,-1.173911,0.871194,0.572105,0.533958,-0.261862,-0.375169,0.084766,1.370113,2.135029,0.579977,-0.729720,2.886699,-1.178918,0.608305,2.495112,1.676402,2.321014,4.193518,-1.294487,0.997253,-0.821323,0.323203,2.299285,-1.903010,-0.114139,1.929224,0.156405,...,-1.238374,2.322325,4.516910,-2.989054,4.985936,0.617126,-0.142685,1.116402,0.428812,-0.518223,-2.938002,1.065818,1.454851,1.201226,-1.471523,-0.328200,-0.595024,-1.843281,-1.528226,-2.343833,0.901847,-1.660347,-1.424284,2.510443,0.105045,0.732640,-1.266058,-0.680594,-0.930322,-1.366645,0.190333,3.219421,0.162525,0.962128,0.638893,-1.211911,-0.331361,-0.640128,0.712659,0.261931
2,-0.391124,0.002255,0.002168,-0.282548,-0.077458,0.397709,-0.104801,-0.223817,0.005838,0.046506,-0.064640,0.766998,-0.084031,0.022012,-0.079032,0.179100,0.083738,0.080692,-0.163746,0.204009,0.430272,0.413277,0.098575,0.078999,0.566312,-0.083736,0.300822,0.472331,0.252414,0.459675,0.687262,-0.164491,0.070970,-0.256913,-0.149491,0.365900,-0.525738,0.070535,0.449914,-0.042546,...,-0.328692,0.356759,0.780401,-0.646330,0.939911,0.174902,0.043133,0.208409,-0.085409,-0.122955,-0.420945,0.150415,0.176600,0.201183,-0.200627,0.062748,-0.036502,-0.402077,-0.325639,-0.350808,0.035271,-0.393972,-0.240491,0.492185,0.043029,0.095770,-0.424533,0.030082,-0.142164,-0.101345,0.003253,0.574001,0.161203,0.212612,0.165526,-0.377995,-0.086006,-0.093979,0.100521,-0.007053
3,-0.914198,-0.139253,0.266807,-0.524489,0.024985,0.695159,-0.148631,-0.728766,-0.099198,0.257817,-0.354547,1.851080,-0.160778,-0.228650,0.282782,0.254939,0.194670,0.000344,-0.208814,0.188393,0.712205,0.811227,0.302672,-0.076347,1.268183,-0.310945,0.704212,1.165475,0.603700,1.183298,1.696786,-0.299544,0.206849,-0.833744,-0.084737,0.739332,-0.737158,0.189926,1.208937,-0.063897,...,-0.480115,1.133498,1.665549,-1.153344,2.197214,0.402356,0.103761,0.169249,0.338988,-0.281748,-0.928820,0.265747,0.744787,0.604130,-0.629640,-0.216055,0.050515,-0.602354,-0.467716,-0.956306,0.461669,-0.874445,-0.480806,1.092533,0.211439,0.278955,-0.473416,-0.207071,-0.253675,-0.542020,0.089284,1.215950,0.675947,0.373924,0.263212,-0.421965,-0.297215,-0.188192,0.209204,0.145678
4,-0.210011,-0.148495,0.183542,-0.109265,-0.056913,0.136753,-0.101321,-0.295220,-0.018304,0.087153,0.045515,0.411938,0.047013,-0.079116,0.087710,0.019022,0.098014,0.022118,-0.019614,-0.064064,0.167425,0.203436,0.049365,0.018488,0.188973,-0.128808,-0.022934,0.251416,0.027218,0.276655,0.294244,-0.057468,0.114947,-0.097524,0.014317,0.194290,-0.133919,-0.069980,0.093464,0.052519,...,-0.184074,0.192396,0.547774,-0.266610,0.478261,0.081075,-0.031807,0.115932,0.035639,-0.016610,-0.155740,0.004228,0.173858,0.168855,-0.145895,-0.087363,-0.051153,-0.104430,-0.090434,-0.228106,0.006348,-0.121624,-0.157486,0.202106,-0.049610,0.057054,-0.234412,-0.029724,-0.095007,-0.152375,0.20458

In [32]:
df_embeddings.to_csv('/content/drive/My Drive/Datasets/df_embeddings.csv')